In [ ]:
###############################################
## THIS NOTEBOOK DOWNLOADS FILES FROM SFDC   ##
##        AND SAVES THEM IN A DRIVE          ##
###############################################

from simple_salesforce import Salesforce
import numpy as np
import pandas as pd
import os
import requests
import re

#clean the filename
def sanitize_filename(filename):
    """Sanitizes a filename by removing or replacing invalid characters."""
    # Replace invalid characters with an underscore or space
    # Customize the regex as needed to keep other valid chars
    # Be careful to not remove too many characters
    sanitized = re.sub(r'[\\/*?:"<>|]', "_", filename)  # Replace with _
    # Remove leading/trailing whitespace
    sanitized = sanitized.strip()
    # Remove or replace characters that might cause problems on certain systems
    sanitized = sanitized.replace(".", "_")  # Avoid double extensions or hidden files etc.
    # Shorten if necessary (some systems have limits)
    max_length = 255  # Or adjust to your needs
    sanitized = sanitized[:max_length]
    return sanitized

## Connect to Salesforce with your username, password, and security token
sf = Salesforce(
    username=os.getenv('SFDC_USER')
    , password=os.getenv('SFDC_PASS')
    , security_token=os.getenv('SFDC_TOKEN')
)

## UDF to query data from SFDC and store in a pandas df
def sf_api_query(soql, dateList=None, tz=None):
    data = sf.query_all(soql)
    try:
        df = pd.DataFrame(data["records"]).drop("attributes", axis=1)
        listColumns = list(df.columns)
        for col in listColumns:
            if any(
                isinstance(df[col].values[i], dict)
                for i in range(0, len(df[col].values))
            ):
                df = pd.concat(
                    [
                        df.drop(columns=[col]),
                        df[col]
                        .apply(pd.Series)
                        .drop("attributes", axis=1)
                        .add_prefix(col + "."),
                    ],
                    axis=1,
                )
                new_columns = np.setdiff1d(df.columns, listColumns)
                for i in new_columns:
                    listColumns.append(i)
        try:
            for date in dateList:
                if max(df[date].str.len()) > 10:
                    try:
                        df[date] = (
                            pd.to_datetime(df[date])
                            .dt.tz_convert(tz)
                            .dt.tz_localize(None)
                        )
                    except:
                        pass
                else:
                    try:
                        df[date] = pd.to_datetime(df[date])
                    except:
                        pass
        except:
            pass
        return df
    except:
        print("The Query returned 0 rows")

## A list of account ids to loop over
##
## You could also query for this
## query = f"""
##        SELECT
##        Id
##        FROM Account 
##        WHERE Industry = 'Manufacturing'
##    """
## accounts = sf_api_query(query)
## account_ids = accounts['Id'].to_list()

account_ids = [
    '0013A00001Rcb7hQAB',
    '0013000000ME4K8AAL'
 ]

## Set a path to store the files
path = r'C:\Users\corey.rastello\SalesforceExtracts\AccountAttachments'

## For each account query their content documents
for id in account_ids:
    query = f"""
        SELECT ContentDocument.Id, 
               ContentDocument.Title,
               ContentDocument.LatestPublishedVersion.Id,
               ContentDocument.FileExtension,
               ContentDocument.LatestPublishedVersion.VersionData,
               LinkedEntityId
        FROM ContentDocumentLink 
        WHERE LinkedEntityId = '{id}'
    """
    content_documents = sf_api_query(query)
    output_dir = os.path.join(path,id)
    os.makedirs(output_dir, exist_ok=True)

    # for each document 
    for index, row in content_documents.iterrows():
        url = f"https://{sf.sf_instance}/services/data/v53.0/sobjects/ContentVersion/{row['ContentDocument.LatestPublishedVersion.Id']}/VersionData"

        headers = {
            "Authorization": f"Bearer {sf.session_id}",
            "User-Agent": "Mozilla/5.0"
        }

        # Step 1: Send the GET request to fetch the file
        response = requests.get(url, headers=headers, stream=True)

        # Step 2: Check the response and save the file if successful
        if response.status_code == 200:
            # Extract the file name (ContentVersion title or custom logic to retrieve it)
            file_name = os.path.join(output_dir,f"{sanitize_filename(row['ContentDocument.Title'])}.{row['ContentDocument.FileExtension']}") 
            
            # Step 3: Save the file to disk
            with open(file_name, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file.write(chunk)
            
            print(f"File downloaded successfully as {file_name}")
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
            print(f"Error: {response.text}")

